In [1]:
import geopandas as gpd
import pandas as pd

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches

pd.options.display.max_columns = 99

## GSheet Connection

In [2]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from numpy import array, nan

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
         './../../../Desktop/newgsheets-349817-e11b8c6a66ec.json', scope) # Your json file here
gc = gspread.authorize(credentials)

sheet_id = '1mtdjdJ_bdvPSP3ojl8bc267A8eYsELRKM16TZjARUSY'
sh = gc.open_by_key(sheet_id)#.get_worksheet('Sheet1')


## Indice de Precios

In [3]:
## INDICE DE PRECIOS - CONVERSION A VALOR CONSTANTE

cpi_M = pd.read_csv('https://raw.githubusercontent.com/matuteiglesias/IPC-Argentina/main/data/info/indice_precios_M.csv', index_col=0)
cpi_M.index = pd.to_datetime(cpi_M.index)

from datetime import datetime as dt
  
# Getting current date and time
today = dt.today()
ref = '2016-01'

cpi_actual = cpi_M.loc[today.strftime("%Y-%m")]['index'][0]
cpi_ref = cpi_M.loc[ref]['index'][0]

# df_ARShoy = df*(cpi_actual/cpi_ref)

cpi_Q = pd.read_csv('https://raw.githubusercontent.com/matuteiglesias/IPC-Argentina/main/data/info/indice_precios_Q.csv', index_col=0)
cpi_Q.index = pd.to_datetime(cpi_Q.index)
cpi_2010Q4 = cpi_Q.loc['2010-11']['index'][0]


### List 'pobreza' files

In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
from IPython.display import display, Markdown, Latex

import glob
frac = 0.01

path ='./../data/Pobreza/pobreza_' # use your path
path ='/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_' # use your path

allFiles = []
allFiles += glob.glob(path+str(frac)+'_q*.csv')
allFiles = sorted(allFiles)
# allFiles[-1]
# allFiles = allFiles[::8]

In [5]:
def moving_avg(table):
    ## Label left. The roilling means are centered nearly on the right date, although one day early.
    # i.e. 2009-12-31 is the average for 07-2009 to 06-2010
    return table.resample('Q', label = 'left').mean().interpolate('linear').rolling(4, center = True).mean()


In [6]:
aglo_labels = pd.read_csv('./../data/info/aglo_labels.csv')[['AGLOMERADO', 'NOMAGLO']]
# main_aglos = ['GBA', 'CABA', 'Cordoba', 'Rosario', 'Mendoza', 'Tucuman', 'La Plata', 'Mar del Plata', 'Salta']
main_aglos = ['GBA', 'CABA', 'Cordoba', 'Rosario', 'Mendoza', 'Tucuman']

In [7]:
# Stats a nivel Hogar
# # Percentile funcs
# def q10(x):
#     return x.quantile(0.1)

# def q25(x):
#     return x.quantile(0.25)

# def q75(x):
#     return x.quantile(0.75)

# def q90(x):
#     return x.quantile(0.9)

# import numpy as np

mon_cols = ['P47T_hogar', 'CBA', 'CBT', 'gap_pobreza', 'gap_indigencia', 'IPCF']

df_list = []

# for f in allFiles[:2]:
# for f in allFiles:
for f in allFiles[-2:]:
    print(f)
    
    vars_ = ['Pobreza', 'Indigencia', 'P47T_persona', 'P0910', 'P03', 'P02', 'Grupo_Etario_INDEC', 'P47T_hogar', 'IX_TOT']
    data = pd.read_csv(f, usecols = ['HOGAR_REF_ID', 'Q', 'AGLOMERADO'] + vars_)
    data['P47T_persona'] = data['P47T_persona']*(cpi_actual/cpi_ref)
    data = data.loc[data.AGLOMERADO != 0]
    data = data.merge(aglo_labels)
    
    ## Grupo Educativo:
    data['Grupo_Educ'] = ''
    data.loc[data.P0910 <= 52, 'Grupo_Educ'] = 'Medio'
    data.loc[data.P0910 > 52, 'Grupo_Educ'] = 'Alto'
    data.loc[(data.P0910 <= 32) | (data.P0910 == 92), 'Grupo_Educ'] = 'Bajo'

    data['NOMAGLO'] = data.NOMAGLO.map(dict(zip(main_aglos, main_aglos))).fillna('Resto de Aglos.')
    df_list += [data]
    
#     data = data.drop_duplicates()
    
#     data['IPCF'] = data['P47T_hogar']/data['IX_TOT']
#     data['IPCF'] = data['IPCF'] + np.random.normal(0, 1e-3, len(data))
    
#     ## Tabla distrib ingreso
#     table = data.groupby(['Q'])[mon_cols].agg(['mean', q10, q25, 'median', q75, q90])
#     df_list += [table]
    
data = pd.concat(df_list)
# df.index = pd.to_datetime(df.index)

/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_q2021-11-15.csv
/media/miglesia/Elements/suite/indice-pobreza-ExactasUBA/data/Pobreza/pobreza_0.01_q2022-02-15.csv


In [8]:
## Tipos de persona
# - generos
# - grupo etario
# - educaion bajo medio alto   (P0910 <= 32 : bajo, 40<= P0910 <= 52, 60<= P0910 <= 82)

parte1 = data.groupby(['P02', 'NOMAGLO'])[['P47T_persona', 'Pobreza', 'Indigencia']].mean().unstack(-1)
parte1.index = ['Varon', 'Mujer']
parte1 = pd.concat([parte1], keys=['Genero'], names=[''])

parte2 = data.groupby(['Grupo_Etario_INDEC', 'NOMAGLO'])[['P47T_persona', 'Pobreza', 'Indigencia']].mean().unstack(-1)
parte2 = pd.concat([parte2], keys=['Grupo Etario'], names=[''])

parte3 = data.groupby(['Grupo_Educ', 'NOMAGLO'])[['P47T_persona', 'Pobreza', 'Indigencia']].mean().unstack(-1)
parte3 = parte3.loc[['Bajo', 'Medio', 'Alto']]
parte3 = pd.concat([parte3], keys=['Nivel Educativo'], names=[''])


In [35]:
tabla = pd.concat([parte1, parte2, parte3])

## Ingresos
tabla1 = tabla['P47T_persona'].round(-2).astype(int)[main_aglos + ['Resto de Aglos.']]
tabla1 = tabla1.drop(('Grupo Etario', '(0, 13]'))

show = (tabla1/1e3).round().astype(int)

show

NOMAGLO                    GBA  CABA  Cordoba  Rosario  Mendoza  Tucuman  \
                                                                           
Genero          Varon       74   125       74       80       66       53   
                Mujer       48    98       52       55       48       40   
Grupo Etario    (13, 29]    43    68       42       44       31       27   
                (29, 64]   102   166      105      107       95       82   
                (64, 110]   91   122       95       91       92       84   
Nivel Educativo Bajo        41    49       34       40       32       27   
                Medio       64    92       58       70       60       48   
                Alto       129   173      113      121      105       94   

NOMAGLO                    Resto de Aglos.  
                                            
Genero          Varon                   67  
                Mujer                   50  
Grupo Etario    (13, 29]                35  
                (29, 64]               101  
                (64, 110]               92  
Nivel Educativo Bajo                    34  
                Medio                   61  
                Alto                   111

In [38]:
## UPDATE G SHEET
input_ =  tabla1.reset_index()
rows, cols = input_.shape

try:
    wks = sh.worksheet('GrEtGenEducAglo_P47T_persona')
except:
    wks = sh.add_worksheet('GrEtGenEducAglo_P47T_persona', 1, 1)

wks.clear()
wks.update([input_.columns.values.tolist()]  + input_.values.tolist())
#     worksheet.update('A1', 'Verdadero')
#     worksheet.update('A2', 'Prediccion')
wks.format('A', {'textFormat': {'bold': True}})
wks.format('B', {'textFormat': {'bold': True}})
wks.format('1', {'textFormat': {'bold': True}})


{'spreadsheetId': '1mtdjdJ_bdvPSP3ojl8bc267A8eYsELRKM16TZjARUSY',
 'replies': [{}]}

In [39]:
# (tabla1.loc['Nivel Educativo'].loc['Alto']/tabla1.loc['Nivel Educativo'].loc['Bajo']).describe()

tabla1.div(tabla1.mean(1), 0).round(2).mean().sort_values()

NOMAGLO
Tucuman            0.74125
Mendoza            0.87000
Resto de Aglos.    0.91375
Cordoba            0.95375
GBA                0.99125
Rosario            1.02250
CABA               1.50250
dtype: float64

In [40]:

# ## Pobreza
tabla2 = 100*tabla['Pobreza'].round(3)[main_aglos + ['Resto de Aglos.']]
tabla2


NOMAGLO                     GBA  CABA  Cordoba  Rosario  Mendoza  Tucuman  \
                                                                            
Genero          Varon      34.7  12.8     31.4     30.7     35.1     39.3   
                Mujer      34.5  12.6     30.8     30.0     34.1     38.3   
Grupo Etario    (0, 13]    52.3  25.8     46.8     50.3     52.2     55.2   
                (13, 29]   40.0  17.7     37.7     37.7     43.2     45.0   
                (29, 64]   26.7   9.5     24.0     23.2     26.9     30.6   
                (64, 110]   7.3   2.0      6.6      5.8      7.1      7.5   
Nivel Educativo Bajo       41.6  20.3     40.0     40.6     42.8     49.1   
                Medio      32.8  14.4     34.1     28.1     34.8     36.3   
                Alto       10.9   5.3     13.5     11.5     16.4     15.1   

NOMAGLO                    Resto de Aglos.  
                                            
Genero          Varon                 35.9  
                Mujer                 34.9  
Grupo Etario    (0, 13]               51.8  
                (13, 29]              43.3  
                (29, 64]              26.8  
                (64, 110]              7.4  
Nivel Educativo Bajo                  44.6  
                Medio                 34.3  
                Alto                  15.6

In [41]:
## UPDATE G SHEET
input_ =  tabla2.reset_index()
rows, cols = input_.shape

try:
    wks = sh.worksheet('GrEtGenEducAglo_Pobreza')
except:
    wks = sh.add_worksheet('GrEtGenEducAglo_Pobreza', 1, 1)

wks.clear()
wks.update([input_.columns.values.tolist()]  + input_.values.tolist())
#     worksheet.update('A1', 'Verdadero')
#     worksheet.update('A2', 'Prediccion')
wks.format('A', {'textFormat': {'bold': True}})
wks.format('B', {'textFormat': {'bold': True}})
wks.format('1', {'textFormat': {'bold': True}})


{'spreadsheetId': '1mtdjdJ_bdvPSP3ojl8bc267A8eYsELRKM16TZjARUSY',
 'replies': [{}]}

In [ ]:

# ## Indigencia
tabla3 = 100*tabla['Indigencia'].round(3)[main_aglos + ['Resto de Aglos.']]
tabla3

In [42]:
## UPDATE G SHEET
input_ =  tabla2.reset_index()
rows, cols = input_.shape

try:
    wks = sh.worksheet('GrEtGenEducAglo_Indigencia')
except:
    wks = sh.add_worksheet('GrEtGenEducAglo_Indigencia', 1, 1)

wks.clear()
wks.update([input_.columns.values.tolist()]  + input_.values.tolist())
#     worksheet.update('A1', 'Verdadero')
#     worksheet.update('A2', 'Prediccion')
wks.format('A', {'textFormat': {'bold': True}})
wks.format('B', {'textFormat': {'bold': True}})
wks.format('1', {'textFormat': {'bold': True}})


{'spreadsheetId': '1mtdjdJ_bdvPSP3ojl8bc267A8eYsELRKM16TZjARUSY',
 'replies': [{}]}

In [43]:
# data.groupby(['Q', 'HOGAR_REF_ID'])

## Contar por genero
cnt_mujeres = data.groupby(['Q', 'HOGAR_REF_ID'])['P02'].apply(lambda x: (x == 2).sum()).reset_index(name='n_mujeres')

# Contar por edad
cnt_adultos = data.groupby(['Q', 'HOGAR_REF_ID'])['P03'].apply(lambda x: (x >= 18).sum()).reset_index(name='n_adultos')
cnt_mayores = data.groupby(['Q', 'HOGAR_REF_ID'])['P03'].apply(lambda x: (x >= 65).sum()).reset_index(name='n_mayores')
cnt_menores = data.groupby(['Q', 'HOGAR_REF_ID'])['P03'].apply(lambda x: (x < 18).sum()).reset_index(name='n_menores')
cnt_m30 = data.groupby(['Q', 'HOGAR_REF_ID'])['P03'].apply(lambda x: (x < 30).sum()).reset_index(name='n_m30')


In [44]:

#genero y edad
data['adulta'] = (data['P02'] == 2)*(data['P03'] >= 18)
cnt_adultas = data.groupby(['Q', 'HOGAR_REF_ID'])['adulta'].apply(lambda x: (x > 0).sum()).reset_index(name='n_adultas')


In [45]:

# Contar por nivel educativo
data['adultalto'] = (data['Grupo_Educ'] == 'Alto')*(data['P03'] >= 18)
cnt_educalto = data.groupby(['Q', 'HOGAR_REF_ID'])['adultalto'].apply(lambda x: (x > 0).sum()).reset_index(name='n_educalto')

data['adultmedio'] = (data['Grupo_Educ'] == 'Medio')*(data['P03'] >= 18)
cnt_educmedio = data.groupby(['Q', 'HOGAR_REF_ID'])['adultmedio'].apply(lambda x: (x > 0).sum()).reset_index(name='n_educmedio')

data['adultbajo'] = (data['Grupo_Educ'] == 'Bajo')*(data['P03'] >= 18)
cnt_educbajo = data.groupby(['Q', 'HOGAR_REF_ID'])['adultbajo'].apply(lambda x: (x > 0).sum()).reset_index(name='n_educbajo')


In [46]:
info = data[['P47T_hogar', 'Q', 'NOMAGLO', 'HOGAR_REF_ID', 'Pobreza', 'Indigencia', 'IX_TOT']].drop_duplicates()

for df in [cnt_mujeres, cnt_adultos, cnt_mayores, cnt_menores, cnt_m30, cnt_educalto, cnt_educmedio, cnt_educbajo, cnt_adultas]:
    info = info.merge(df)

In [47]:

info['Tipo_Hogar'] = ''
# info.loc[(info.IX_TOT == 1) & (info.n_mujeres == 1), 'Tipo_Hogar'] = 'Mujer'
# info.loc[(info.IX_TOT == 1) & (info.n_mujeres == 0), 'Tipo_Hogar'] = 'Varon'

info.loc[(info.IX_TOT == 1) & (info.n_mujeres == 1) & (info.n_m30 == 1), 'Tipo_Hogar'] = 'Mujer menor de 30'
info.loc[(info.IX_TOT == 1) & (info.n_mujeres == 1) & (info.n_m30 == 0) & (info.n_mayores == 0), 'Tipo_Hogar'] = 'Mujer, entre 30 y 65'
info.loc[(info.IX_TOT == 1) & (info.n_mujeres == 1) & (info.n_mayores == 1), 'Tipo_Hogar'] = 'Mujer, mayor de 65'
info.loc[(info.IX_TOT == 1) & (info.n_mujeres == 0) & (info.n_m30 == 1), 'Tipo_Hogar'] = 'Varon menor de 30'
info.loc[(info.IX_TOT == 1) & (info.n_mujeres == 0) & (info.n_m30 == 0) & (info.n_mayores == 0), 'Tipo_Hogar'] = 'Varon entre 30 y 65'
info.loc[(info.IX_TOT == 1) & (info.n_mujeres == 0) & (info.n_mayores == 1), 'Tipo_Hogar'] = 'Varon mayor de 65'


info.loc[(info.IX_TOT == 2) & (info.n_menores == 0) & (info.n_mayores >= 1), 'Tipo_Hogar'] = '2 adultos, al menos uno +65'
info.loc[(info.IX_TOT == 2) & (info.n_menores == 0) & (info.n_mayores == 0), 'Tipo_Hogar'] = '2 adultos, ninguno +65'
info.loc[(info.IX_TOT > 2) & (info.n_menores == 0), 'Tipo_Hogar'] = 'Mas de 2 adultos'

info.loc[(info.IX_TOT >= 2) & (info.n_menores >= 1) & (info.n_adultos == 1), 'Tipo_Hogar'] = 'Padre soltero con menores'
info.loc[(info.IX_TOT >= 2) & (info.n_menores >= 1) & (info.n_adultos == 1) & (info.n_adultas > 0), 'Tipo_Hogar'] = 'Madre soltera con menores'
info.loc[(info.IX_TOT == 3) & (info.n_menores == 1) & (info.n_adultos == 2), 'Tipo_Hogar'] = '2 adultos y un menor'
info.loc[(info.IX_TOT == 4) & (info.n_menores == 2) & (info.n_adultos == 2), 'Tipo_Hogar'] = '2 adultos y 2 menores'
info.loc[(info.IX_TOT >= 5) & (info.n_menores >= 3) & (info.n_adultos == 2), 'Tipo_Hogar'] = '2 adultos y 3 o mas menores'
info.loc[(info.IX_TOT > 2) & (info.n_menores >= 1) & (info.n_adultos >= 2), 'Tipo_Hogar'] = 'Al menos 2 adultos y al menos 1 menor'

info['Tamano_Hogar'] = (info.IX_TOT > 1).map({False: '1 pers.', True: '2 o mas pers.'})


In [48]:

info['Educ_Hogar'] = ''

## El orden es importante para que no se sobreescriban las condiciones.
info.loc[(info.n_educmedio >= 1), 'Educ_Hogar'] = 'Al menos un adulto Medio' # ok
info.loc[(info.n_educalto >= 1), 'Educ_Hogar'] = 'Al menos un adulto Alto' # ok
info.loc[(info.n_adultos == info.n_educmedio), 'Educ_Hogar'] = 'Todos los adultos Medio'
info.loc[(info.n_adultos == info.n_educalto), 'Educ_Hogar'] = 'Todos los adultos Alto'
info.loc[(info.n_adultos == info.n_educbajo), 'Educ_Hogar'] = 'Todos los adultos Bajo' # ok


In [49]:
cnt_tipo_hogar = info.groupby(['Tamano_Hogar', 'Tipo_Hogar']).agg({'IX_TOT': ['sum', 'mean', 'size']})['IX_TOT']
# cnt_tipo_hogar['test'] = cnt_tipo_hogar['mean']*cnt_tipo_hogar['size']
cnt_tipo_hogar[['sum', 'size']] = (cnt_tipo_hogar[['sum', 'size']]/frac/2/1e3).round().astype(int)
cnt_tipo_hogar.columns = ['n_personas (mil)', 't_hogar', 'n_hogares (mil)']


In [50]:
cnt_educ_hogar = info.groupby(['Educ_Hogar']).agg({'IX_TOT': ['sum', 'mean', 'size']})['IX_TOT']
# cnt_tipo_hogar['test'] = cnt_tipo_hogar['mean']*cnt_tipo_hogar['size']
cnt_educ_hogar[['sum', 'size']] = (cnt_educ_hogar[['sum', 'size']]/frac/2/1e3).round().astype(int)
cnt_educ_hogar.columns = ['n_personas (mil)', 't_hogar', 'n_hogares (mil)']


In [52]:
table = 100*info.groupby(['Tamano_Hogar', 'Tipo_Hogar', 'NOMAGLO'])['Pobreza'].mean().unstack()

show = pd.concat([table.round(1), cnt_tipo_hogar[['n_hogares (mil)', 'n_personas (mil)']].round(2)], axis = 1)
show = show.loc[show.index.get_level_values('Tipo_Hogar') != '']
show

CABA  Cordoba   GBA  \
Tamano_Hogar  Tipo_Hogar                                                   
1 pers.       Mujer menor de 30                      16.7     24.3  21.4   
              Mujer, entre 30 y 65                    6.1      7.0  13.8   
              Mujer, mayor de 65                      0.5      0.2   2.5   
              Varon entre 30 y 65                     7.4     11.5  14.5   
              Varon mayor de 65                       1.9      7.6   5.9   
              Varon menor de 30                      13.8     20.6  25.9   
2 o mas pers. 2 adultos, al menos uno +65             1.9      3.7   4.5   
              2 adultos, ninguno +65                  7.2     16.8  12.4   
              Al menos 2 adultos y al menos 1 menor  17.8     36.4  40.2   
              Madre soltera con menores              38.3     55.9  63.1   
              Mas de 2 adultos                        5.2     11.3   9.3   
              Padre soltero con menores              27.1     60.7  52.7   

                                                     Mendoza  Resto de Aglos.  \
Tamano_Hogar  Tipo_Hogar                                                        
1 pers.       Mujer menor de 30                         28.6             25.4   
              Mujer, entre 30 y 65                      10.1             10.1   
              Mujer, mayor de 65                         0.8              0.5   
              Varon entre 30 y 65                       17.3             14.9   
              Varon mayor de 65                          6.7              5.9   
              Varon menor de 30                         48.9             28.8   
2 o mas pers. 2 adultos, al menos uno +65                2.9              4.4   
              2 adultos, ninguno +65                    14.5             15.6   
              Al menos 2 adultos y al menos 1 menor     42.2             40.4   
              Madre soltera con menores                 65.1             58.1   
              Mas de 2 adultos                          13.0             12.6   
              Padre soltero con menores                 56.8             51.9   

                                                     Rosario  Tucuman  \
Tamano_Hogar  Tipo_Hogar                                                
1 pers.       Mujer menor de 30                         24.8     10.2   
              Mujer, entre 30 y 65                       9.6     10.9   
              Mujer, mayor de 65                         0.3      0.0   
              Varon entre 30 y 65                        9.7     13.0   
              Varon mayor de 65                          4.5      3.1   
              Varon menor de 30                         23.3     39.0   
2 o mas pers. 2 adultos, al menos uno +65                4.6      3.6   
              2 adultos, ninguno +65                    16.1     20.0   
              Al menos 2 adultos y al menos 1 menor     38.1     43.8   
              Madre soltera con menores                 66.2     62.1   
              Mas de 2 adultos                          11.0     12.3   
              Padre soltero con menores                 55.8     70.7   

                                                     n_hogares (mil)  \
Tamano_Hogar  Tipo_Hogar                                               
1 pers.       Mujer menor de 30                                   91   
              Mujer, entre 30 y 65                               341   
              Mujer, mayor de 65                                 433   
              Varon entre 30 y 65                                391   
              Varon mayor de 65                                  149   
              Varon menor de 30                                  116   
2 o mas pers. 2 adultos, al menos uno +65                        738   
              2 adultos, ninguno +65                            1049   
              Al menos 2 adultos y al menos 1 menor             3745   
              Madre soltera con menore

In [53]:
## UPDATE G SHEET
input_ =  show.reset_index()
rows, cols = input_.shape

try:
    wks = sh.worksheet('TipoHogar_Pobreza')
except:
    wks = sh.add_worksheet('TipoHogar_Pobreza', 1, 1)

wks.clear()
wks.update([input_.columns.values.tolist()]  + input_.values.tolist())
#     worksheet.update('A1', 'Verdadero')
#     worksheet.update('A2', 'Prediccion')
wks.format('A', {'textFormat': {'bold': True}})
wks.format('B', {'textFormat': {'bold': True}})
wks.format('1', {'textFormat': {'bold': True}})


{'spreadsheetId': '1mtdjdJ_bdvPSP3ojl8bc267A8eYsELRKM16TZjARUSY',
 'replies': [{}]}

In [55]:
table = 100*info.groupby(['Tamano_Hogar', 'Tipo_Hogar', 'NOMAGLO'])['Indigencia'].mean().unstack().round(3)

show = pd.concat([table.round(1), cnt_tipo_hogar[['n_hogares (mil)', 'n_personas (mil)']].round(2)], axis = 1)
show = show.loc[show.index.get_level_values('Tipo_Hogar') != '']
show

CABA  Cordoba   GBA  \
Tamano_Hogar  Tipo_Hogar                                                   
1 pers.       Mujer menor de 30                       9.1      5.5  13.4   
              Mujer, entre 30 y 65                    2.5      2.3   5.5   
              Mujer, mayor de 65                      0.3      0.0   1.7   
              Varon entre 30 y 65                     2.5      2.7   4.4   
              Varon mayor de 65                       0.9      0.0   1.7   
              Varon menor de 30                       5.8      3.8  11.5   
2 o mas pers. 2 adultos, al menos uno +65             0.1      0.1   0.1   
              2 adultos, ninguno +65                  3.0      6.4   3.4   
              Al menos 2 adultos y al menos 1 menor   3.6      6.6   7.5   
              Madre soltera con menores              15.1     28.4  29.6   
              Mas de 2 adultos                        1.4      2.7   1.1   
              Padre soltero con menores              11.4     18.0  17.7   

                                                     Mendoza  Resto de Aglos.  \
Tamano_Hogar  Tipo_Hogar                                                        
1 pers.       Mujer menor de 30                         11.4              9.6   
              Mujer, entre 30 y 65                       2.6              3.2   
              Mujer, mayor de 65                         0.0              0.1   
              Varon entre 30 y 65                        2.0              4.7   
              Varon mayor de 65                          0.0              0.3   
              Varon menor de 30                          8.5             11.8   
2 o mas pers. 2 adultos, al menos uno +65                0.0              0.1   
              2 adultos, ninguno +65                     4.0              4.9   
              Al menos 2 adultos y al menos 1 menor      9.0              8.1   
              Madre soltera con menores                 32.3             29.2   
              Mas de 2 adultos                           2.4              2.4   
              Padre soltero con menores                 27.0             18.5   

                                                     Rosario  Tucuman  \
Tamano_Hogar  Tipo_Hogar                                                
1 pers.       Mujer menor de 30                         14.3      8.2   
              Mujer, entre 30 y 65                       2.0      9.4   
              Mujer, mayor de 65                         0.0      0.0   
              Varon entre 30 y 65                        4.4      6.2   
              Varon mayor de 65                          0.0      0.0   
              Varon menor de 30                         12.8     13.6   
2 o mas pers. 2 adultos, al menos uno +65                0.0      0.0   
              2 adultos, ninguno +65                     6.5      4.9   
              Al menos 2 adultos y al menos 1 menor      7.7     10.9   
              Madre soltera con menores                 36.5     32.5   
              Mas de 2 adultos                           1.6      2.8   
              Padre soltero con menores                 22.1     36.6   

                                                     n_hogares (mil)  \
Tamano_Hogar  Tipo_Hogar                                               
1 pers.       Mujer menor de 30                                   91   
              Mujer, entre 30 y 65                               341   
              Mujer, mayor de 65                                 433   
              Varon entre 30 y 65                                391   
              Varon mayor de 65                                  149   
              Varon menor de 30                                  116   
2 o mas pers. 2 adultos, al menos uno +65                        738   
              2 adultos, ninguno +65                            1049   
              Al menos 2 adultos y al menos 1 menor             3745   
              Madre soltera con menore

In [56]:
## UPDATE G SHEET
input_ =  show.reset_index()
rows, cols = input_.shape

try:
    wks = sh.worksheet('TipoHogar_Indigencia')
except:
    wks = sh.add_worksheet('TipoHogar_Indigencia', 1, 1)

wks.clear()
wks.update([input_.columns.values.tolist()]  + input_.values.tolist())
#     worksheet.update('A1', 'Verdadero')
#     worksheet.update('A2', 'Prediccion')
wks.format('A', {'textFormat': {'bold': True}})
wks.format('B', {'textFormat': {'bold': True}})
wks.format('1', {'textFormat': {'bold': True}})


{'spreadsheetId': '1mtdjdJ_bdvPSP3ojl8bc267A8eYsELRKM16TZjARUSY',
 'replies': [{}]}

In [63]:
info['P47T_hogar_ARSh'] = info['P47T_hogar']*(cpi_actual/cpi_ref)

table = info.groupby(['Tamano_Hogar', 'Tipo_Hogar', 'NOMAGLO'])['P47T_hogar_ARSh'].mean().unstack().round(3)

show = pd.concat([(table/1e3).round(1), cnt_tipo_hogar.round(2)], axis = 1)
show = show.loc[show.index.get_level_values('Tipo_Hogar') != '']
show

CABA  Cordoba    GBA  \
Tamano_Hogar  Tipo_Hogar                                                     
1 pers.       Mujer menor de 30                      128.8     89.1   92.9   
              Mujer, entre 30 y 65                   172.0    121.8  107.4   
              Mujer, mayor de 65                     116.6     97.9   88.7   
              Varon entre 30 y 65                    186.9    130.3  133.5   
              Varon mayor de 65                      139.6    148.3   94.6   
              Varon menor de 30                      132.8     88.8   92.4   
2 o mas pers. 2 adultos, al menos uno +65            262.4    181.7  189.6   
              2 adultos, ninguno +65                 307.2    187.3  210.0   
              Al menos 2 adultos y al menos 1 menor  349.2    225.5  218.4   
              Madre soltera con menores              132.3     91.8   80.2   
              Mas de 2 adultos                       479.5    305.6  325.6   
              Padre soltero con menores              179.4     87.6  113.1   

                                                     Mendoza  Resto de Aglos.  \
Tamano_Hogar  Tipo_Hogar                                                        
1 pers.       Mujer menor de 30                         84.1             78.0   
              Mujer, entre 30 y 65                     107.6            109.3   
              Mujer, mayor de 65                       100.0             95.8   
              Varon entre 30 y 65                      131.0            127.6   
              Varon mayor de 65                         99.1             98.8   
              Varon menor de 30                         74.8             80.7   
2 o mas pers. 2 adultos, al menos uno +65              185.0            186.4   
              2 adultos, ninguno +65                   179.8            197.6   
              Al menos 2 adultos y al menos 1 menor    200.7            215.3   
              Madre soltera con menores                 74.9             84.4   
              Mas de 2 adultos                         302.4            302.9   
              Padre soltero con menores                122.5            102.5   

                                                     Rosario  Tucuman  \
Tamano_Hogar  Tipo_Hogar                                                
1 pers.       Mujer menor de 30                         93.1     63.5   
              Mujer, entre 30 y 65                     118.7     94.3   
              Mujer, mayor de 65                        91.6    105.5   
              Varon entre 30 y 65                      136.6    115.4   
              Varon mayor de 65                         98.3    106.3   
              Varon menor de 30                         93.5     76.9   
2 o mas pers. 2 adultos, al menos uno +65              180.0    174.8   
              2 adultos, ninguno +65                   196.1    159.7   
              Al menos 2 adultos y al menos 1 menor    226.8    184.9   
              Madre soltera con menores                 73.1     68.6   
              Mas de 2 adultos                         311.4    262.1   
              Padre soltero con menores                 98.6     68.5   

                                                     n_personas (mil)  \
Tamano_Hogar  Tipo_Hogar                                                
1 pers.       Mujer menor de 30                                    91   
              Mujer, entre 30 y 65                                341   
              Mujer, mayor de 65                                  433   
              Varon entre 30 y 65                                 391   
              Varon mayor de 65                                   149   
              Varon menor de 30                                   116   
2 o mas pers. 2 adultos, al menos uno +65                        1477   
              2 adultos, ninguno +65                             2098   
              Al menos 2 adultos y al menos 1 menor             17677   


In [64]:
## UPDATE G SHEET
input_ =  show.reset_index()
rows, cols = input_.shape

try:
    wks = sh.worksheet('TipoHogar_P47T_hogar_ARSh')
except:
    wks = sh.add_worksheet('TipoHogar_P47T_hogar_ARSh', 1, 1)

wks.clear()
wks.update([input_.columns.values.tolist()]  + input_.values.tolist())
#     worksheet.update('A1', 'Verdadero')
#     worksheet.update('A2', 'Prediccion')
wks.format('A', {'textFormat': {'bold': True}})
wks.format('B', {'textFormat': {'bold': True}})
wks.format('1', {'textFormat': {'bold': True}})


{'spreadsheetId': '1mtdjdJ_bdvPSP3ojl8bc267A8eYsELRKM16TZjARUSY',
 'replies': [{}]}

In [65]:
table = 100*info.groupby(['Educ_Hogar', 'NOMAGLO'])['Pobreza'].mean().unstack()

show = pd.concat([table.round(1), cnt_educ_hogar[['n_hogares (mil)', 'n_personas (mil)']].round(2)], axis = 1).sort_values('Resto de Aglos.')
show

,CABA,Cordoba,GBA,Mendoza,Resto de Aglos.,Rosario,Tucuman,n_hogares (mil),n_personas (mil)
Educ_Hogar,,,,,,,,,
Todos los adultos Alto,5.4,12.4,8.2,12.8,13.3,9.8,12.3,1608,3878
Al menos un adulto Alto,6.7,15.9,14.9,21.4,19.1,14.4,19.1,1938,7658
Todos los adultos Medio,14.7,32.5,31.3,36.9,32.8,25.6,36.0,1641,4615
Todos los adultos Bajo,14.4,30.5,32.7,31.5,34.9,31.9,46.3,1766,4823
Al menos un adulto Medio,18.6,38.7,35.0,37.9,37.9,34.3,44.4,1764,7554


In [66]:
## UPDATE G SHEET
input_ =  show.reset_index()
rows, cols = input_.shape

try:
    wks = sh.worksheet('Educ_Hogar_Pobreza')
except:
    wks = sh.add_worksheet('Educ_Hogar_Pobreza', 1, 1)

wks.clear()
wks.update([input_.columns.values.tolist()]  + input_.values.tolist())

wks.format('A', {'textFormat': {'bold': True}})
# wks.format('B', {'textFormat': {'bold': True}})
wks.format('1', {'textFormat': {'bold': True}})


{'spreadsheetId': '1mtdjdJ_bdvPSP3ojl8bc267A8eYsELRKM16TZjARUSY',
 'replies': [{}]}

In [67]:
table = 100*info.groupby(['Educ_Hogar', 'NOMAGLO'])['Indigencia'].mean().unstack().round(3)

show = pd.concat([table.round(1), cnt_educ_hogar[['n_hogares (mil)', 'n_personas (mil)']].round(2)], axis = 1).sort_values('Resto de Aglos.')
show

,CABA,Cordoba,GBA,Mendoza,Resto de Aglos.,Rosario,Tucuman,n_hogares (mil),n_personas (mil)
Educ_Hogar,,,,,,,,,
Al menos un adulto Alto,1.2,2.6,2.1,3.0,3.4,2.1,3.7,1938,7658
Todos los adultos Alto,2.1,4.0,2.3,3.1,4.2,4.3,4.4,1608,3878
Al menos un adulto Medio,3.9,7.3,6.5,8.5,7.4,6.8,10.0,1764,7554
Todos los adultos Medio,4.6,8.8,8.3,10.3,9.0,6.8,11.1,1641,4615
Todos los adultos Bajo,5.0,8.2,9.2,9.2,10.7,10.7,16.7,1766,4823


In [68]:
## UPDATE G SHEET
input_ =  show.reset_index()
rows, cols = input_.shape

try:
    wks = sh.worksheet('Educ_Hogar_Indigencia')
except:
    wks = sh.add_worksheet('Educ_Hogar_Indigencia', 1, 1)

wks.clear()
wks.update([input_.columns.values.tolist()]  + input_.values.tolist())

wks.format('A', {'textFormat': {'bold': True}})
# wks.format('B', {'textFormat': {'bold': True}})
wks.format('1', {'textFormat': {'bold': True}})


{'spreadsheetId': '1mtdjdJ_bdvPSP3ojl8bc267A8eYsELRKM16TZjARUSY',
 'replies': [{}]}

In [69]:
info['P47T_hogar_ARSh'] = info['P47T_hogar']*(cpi_actual/cpi_ref)

table = info.groupby(['Educ_Hogar', 'NOMAGLO'])['P47T_hogar_ARSh'].mean().unstack()

show = pd.concat([(table/1e3).round(1), cnt_educ_hogar[['n_hogares (mil)', 'n_personas (mil)']].round(2)], axis = 1).sort_values('Resto de Aglos.')
show

,CABA,Cordoba,GBA,Mendoza,Resto de Aglos.,Rosario,Tucuman,n_hogares (mil),n_personas (mil)
Educ_Hogar,,,,,,,,,
Todos los adultos Bajo,136.7,122.1,136.1,120.5,120.5,122.6,105.7,1766,4823
Todos los adultos Medio,176.1,142.0,155.7,137.0,146.5,155.0,123.4,1641,4615
Al menos un adulto Medio,268.3,201.7,216.7,184.7,194.3,203.6,169.2,1764,7554
Todos los adultos Alto,308.3,222.2,275.8,233.9,226.0,237.5,219.8,1608,3878
Al menos un adulto Alto,406.9,268.2,297.5,265.1,275.5,282.9,240.9,1938,7658


In [70]:
## UPDATE G SHEET
input_ =  show.reset_index()
rows, cols = input_.shape

try:
    wks = sh.worksheet('Educ_Hogar_P47T_hogar_ARSh')
except:
    wks = sh.add_worksheet('Educ_Hogar_P47T_hogar_ARSh', 1, 1)

wks.clear()
wks.update([input_.columns.values.tolist()]  + input_.values.tolist())

wks.format('A', {'textFormat': {'bold': True}})
# wks.format('B', {'textFormat': {'bold': True}})
wks.format('1', {'textFormat': {'bold': True}})


{'spreadsheetId': '1mtdjdJ_bdvPSP3ojl8bc267A8eYsELRKM16TZjARUSY',
 'replies': [{}]}